# Setup

In [ ]:
%pip install -r requirements.txt

In [31]:
import torch 
import argparse
from utils import dotdict
from activation_dataset import setup_token_data
import wandb
import json
from datetime import datetime
from tqdm import tqdm
from einops import rearrange
import matplotlib.pyplot as plt
import os
from baukit import Trace, TraceDict
import torch.nn as nn
from autoencoders.learned_dict import TiedSAE, UntiedSAE, AnthropicSAE

def init_cfg():
    cfg = dotdict()
    # models: "EleutherAI/pythia-6.9b", "lomahony/eleuther-pythia6.9b-hh-sft", "usvsnsp/pythia-6.9b-ppo", "Dahoas/gptj-rm-static", "reciprocate/dahoas-gptj-rm-static"
    # cfg.model_name="lomahony/eleuther-pythia6.9b-hh-sft"
    # "EleutherAI/pythia-70m", "lomahony/pythia-70m-helpful-sft", "lomahony/eleuther-pythia70m-hh-sft"
    cfg.model_name="EleutherAI/pythia-70m-deduped"
    cfg.layers=[0, 1] # Change this to run multiple layers
    cfg.setting="residual"
    # cfg.tensor_name="gpt_neox.layers.{layer}" or "transformer.h.{layer}"
    cfg.tensor_name="gpt_neox.layers.{layer}.mlp"
    original_l1_alpha = 8e-4
    cfg.l1_alpha=original_l1_alpha
    cfg.l1_alphas=[8e-5, 1e-4, 2e-4, 4e-4, 8e-4, 1e-3, 2e-3, 4e-3, 8e-3]
    cfg.sparsity=None
    cfg.num_epochs=2
    cfg.model_batch_size=8
    cfg.lr=1e-3 # ORIGINAL: 1e-3
    cfg.kl=False
    cfg.reconstruction=False
    #cfg.dataset_name="NeelNanda/pile-10k"
    cfg.dataset_name="Elriggs/openwebtext-100k"
    cfg.device="cuda:0"
    cfg.ratio = 8
    cfg.seed = 0
    # cfg.device="cpu"

    return cfg

# Main Code

In [35]:
def setup_execute_training(model_name,
                          dataset_name,
                          ratio,
                          layers,
                          seed,
                          wandb_log,
                          split,
                          epoches):
    cfg = init_cfg()
    cfg.num_epochs = epoches
    cfg.model_name = model_name
    cfg.dataset_name = dataset_name
    cfg.ratio = ratio
    cfg.layers = layers
    cfg.seed = seed
    cfg.wandb_log = wandb_log

    model, tokenizer = load_model(cfg)
    get_activation_size(cfg, model, tokenizer)

    # naming
    start_time = datetime.now().strftime("%Y%m%d-%H%M%S")
    wandb_run_name = f"{cfg.model_name}_{cfg.dataset_name}_s{cfg.seed}_dim{cfg.ratio*cfg.activation_size}_{start_time[4:]}"
    model_name_path = cfg.model_name.replace("/", "_")
    dataset_name_path = cfg.dataset_name.split("/")[-1]
    storage_path = f"{model_name_path}/{dataset_name_path}_s{cfg.seed}"
    filename = f"{cfg.ratio*cfg.activation_size}_{start_time[4:]}"
    token_loader = init_dataloader(cfg, model, tokenizer, split)
    autoencoders, optimizers = init_autoencoder(cfg)
    
    if wandb_log:
        setup_wandb(cfg, wandb_run_name)
    
    training_run(cfg, model, optimizers, autoencoders, token_loader)

    for layer in range(len(cfg.layers)):
        model_save(cfg, autoencoders[layer], storage_path, filename, cfg.layers[layer])

In [36]:
# Code that actually starts a full training run!

model_name = "EleutherAI/pythia-160m"
dataset_name = "Elriggs/openwebtext-100k" # "Elriggs/openwebtext-100k"
ratio = 32
layers = [0, 1, 2, 3, 4, 5]
wandb_log = False
seed = 0
split = "train"
epoches = 1

setup_execute_training(model_name,
                       dataset_name,
                       ratio,
                       layers,
                       seed,
                       wandb_log=wandb_log,
                       split=split,
                      epoches=epoches)

KeyboardInterrupt: 

In [39]:
# Code that actually starts a full training run!

model_name = "EleutherAI/pythia-160m"
dataset_name = "Elriggs/openwebtext-100k" # "Elriggs/openwebtext-100k"
ratio = 16
layers = [0, 1, 2, 3, 4, 5]
wandb_log = False
seed = 0
split = "train"
epoches = 1

setup_execute_training(model_name,
                       dataset_name,
                       ratio,
                       layers,
                       seed,
                       wandb_log=wandb_log,
                       split=split,
                      epoches=epoches)

Activation size: 512


Found cached dataset parquet (/root/.cache/huggingface/datasets/NeelNanda___parquet/NeelNanda--pile-10k-72f566e9f7c464ab/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Loading cached processed dataset at /root/.cache/huggingface/datasets/NeelNanda___parquet/NeelNanda--pile-10k-72f566e9f7c464ab/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-c8791bdc71185487_*_of_00008.arrow


Number of tokens: 2441216


  0%|          | 3/1192 [00:00<00:47, 24.81it/s]

Sparsity: 4090.7 | Dead Features: 8192 | Total Loss: 0.3287 | Reconstruction Loss: 0.1124 | L1 Loss: 0.2163 | l1_alpha: 8.0000e-04 | Tokens: 0 | Self Similarity: 1.0000
Sparsity: 4085.1 | Dead Features: 8192 | Total Loss: 0.2414 | Reconstruction Loss: 0.0772 | L1 Loss: 0.1643 | l1_alpha: 8.0000e-04 | Tokens: 0 | Self Similarity: -0.0003
Sparsity: 4072.4 | Dead Features: 8192 | Total Loss: 0.8384 | Reconstruction Loss: 0.6003 | L1 Loss: 0.2382 | l1_alpha: 8.0000e-04 | Tokens: 0 | Self Similarity: -0.0009
Sparsity: 4102.5 | Dead Features: 8192 | Total Loss: 0.6509 | Reconstruction Loss: 0.2983 | L1 Loss: 0.3525 | l1_alpha: 8.0000e-04 | Tokens: 0 | Self Similarity: 0.0004
Sparsity: 4076.5 | Dead Features: 8192 | Total Loss: 1.1337 | Reconstruction Loss: 0.6610 | L1 Loss: 0.4727 | l1_alpha: 8.0000e-04 | Tokens: 0 | Self Similarity: 0.0015
Sparsity: 4070.8 | Dead Features: 8192 | Total Loss: 9.9328 | Reconstruction Loss: 8.1406 | L1 Loss: 1.7922 | l1_alpha: 8.0000e-04 | Tokens: 0 | Self Sim

...

Sparsity: 39.9 | Dead Features: 0 | Total Loss: 0.0119 | Reconstruction Loss: 0.0050 | L1 Loss: 0.0068 | l1_alpha: 8.0000e-04 | Tokens: 2048000 | Self Similarity: -0.0034
Sparsity: 63.4 | Dead Features: 0 | Total Loss: 0.0153 | Reconstruction Loss: 0.0092 | L1 Loss: 0.0061 | l1_alpha: 8.0000e-04 | Tokens: 2048000 | Self Similarity: -0.0105
Sparsity: 72.1 | Dead Features: 0 | Total Loss: 0.0227 | Reconstruction Loss: 0.0136 | L1 Loss: 0.0091 | l1_alpha: 8.0000e-04 | Tokens: 2048000 | Self Similarity: 0.0649
Sparsity: 143.8 | Dead Features: 0 | Total Loss: 0.0451 | Reconstruction Loss: 0.0217 | L1 Loss: 0.0234 | l1_alpha: 8.0000e-04 | Tokens: 2048000 | Self Similarity: 0.0289
Sparsity: 165.4 | Dead Features: 0 | Total Loss: 0.0601 | Reconstruction Loss: 0.0301 | L1 Loss: 0.0300 | l1_alpha: 8.0000e-04 | Tokens: 2048000 | Self Similarity: -0.0057
Sparsity: 132.5 | Dead Features: 0 | Total Loss: 0.1736 | Reconstruction Loss: 0.1134 | L1 Loss: 0.0602 | l1_alpha: 8.0000e-04 | Tokens: 2048000 

 93%|█████████▎| 1104/1192 [00:43<00:03, 24.88it/s]

Sparsity: 34.7 | Dead Features: 0 | Total Loss: 0.0106 | Reconstruction Loss: 0.0042 | L1 Loss: 0.0065 | l1_alpha: 8.0000e-04 | Tokens: 2252800 | Self Similarity: -0.0035
Sparsity: 59.9 | Dead Features: 0 | Total Loss: 0.0145 | Reconstruction Loss: 0.0087 | L1 Loss: 0.0058 | l1_alpha: 8.0000e-04 | Tokens: 2252800 | Self Similarity: -0.0098
Sparsity: 69.2 | Dead Features: 0 | Total Loss: 0.0214 | Reconstruction Loss: 0.0127 | L1 Loss: 0.0087 | l1_alpha: 8.0000e-04 | Tokens: 2252800 | Self Similarity: 0.0666
Sparsity: 141.1 | Dead Features: 0 | Total Loss: 0.0431 | Reconstruction Loss: 0.0209 | L1 Loss: 0.0222 | l1_alpha: 8.0000e-04 | Tokens: 2252800 | Self Similarity: 0.0295
Sparsity: 159.0 | Dead Features: 0 | Total Loss: 0.0568 | Reconstruction Loss: 0.0289 | L1 Loss: 0.0279 | l1_alpha: 8.0000e-04 | Tokens: 2252800 | Self Similarity: -0.0056
Sparsity: 81.6 | Dead Features: 0 | Total Loss: 0.1599 | Reconstruction Loss: 0.1035 | L1 Loss: 0.0564 | l1_alpha: 8.0000e-04 | Tokens: 2252800 |

100%|██████████| 1192/1192 [00:47<00:00, 25.28it/s]


Saved file at: trained_models/EleutherAI_pythia-70m/pile-10k_s0/layer_0/L0_8192_1212-175906.pt
Saved file at: trained_models/EleutherAI_pythia-70m/pile-10k_s0/layer_1/L1_8192_1212-175906.pt
Saved file at: trained_models/EleutherAI_pythia-70m/pile-10k_s0/layer_2/L2_8192_1212-175906.pt
Saved file at: trained_models/EleutherAI_pythia-70m/pile-10k_s0/layer_3/L3_8192_1212-175906.pt
Saved file at: trained_models/EleutherAI_pythia-70m/pile-10k_s0/layer_4/L4_8192_1212-175906.pt
Saved file at: trained_models/EleutherAI_pythia-70m/pile-10k_s0/layer_5/L5_8192_1212-175906.pt


In [ ]:
# Code that actually starts a full training run!

model_name = "EleutherAI/pythia-160m"
dataset_name = "Elriggs/openwebtext-100k" # "Elriggs/openwebtext-100k"
ratio = 2
layers = [0, 1, 2, 3, 4, 5]
wandb_log = False
seed = 0
split = "train"
epoches = 1

setup_execute_training(model_name,
                       dataset_name,
                       ratio,
                       layers,
                       seed,
                       wandb_log=wandb_log,
                       split=split,
                      epoches=epoches)

In [29]:
# Code that actually starts a full training run!

model_name = "EleutherAI/pythia-70m"
dataset_name = "Elriggs/openwebtext-100k" # "Elriggs/openwebtext-100k"
ratio = 2
layers = [0, 1, 2, 3, 4, 5]
wandb_log = False
seed = 0
split = "train"
epoches = 1

setup_execute_training(model_name,
                       dataset_name,
                       ratio,
                       layers,
                       seed,
                       wandb_log=wandb_log,
                       split=split,
                      epoches=epoches)

Activation size: 512


Found cached dataset parquet (/root/.cache/huggingface/datasets/Elriggs___parquet/Elriggs--openwebtext-100k-79076ecafee8a6d5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Loading cached processed dataset at /root/.cache/huggingface/datasets/Elriggs___parquet/Elriggs--openwebtext-100k-79076ecafee8a6d5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-d3de196f0681d12e_*_of_00008.arrow


Number of tokens: 112750592


  0%|          | 2/55054 [00:00<47:23, 19.36it/s]

Sparsity: 508.8 | Dead Features: 1024 | Total Loss: 0.1624 | Reconstruction Loss: 0.0987 | L1 Loss: 0.0637 | l1_alpha: 8.0000e-04 | Tokens: 0 | Self Similarity: 1.0000
Sparsity: 507.4 | Dead Features: 1024 | Total Loss: 0.1081 | Reconstruction Loss: 0.0615 | L1 Loss: 0.0465 | l1_alpha: 8.0000e-04 | Tokens: 0 | Self Similarity: -0.0004
Sparsity: 511.9 | Dead Features: 1024 | Total Loss: 0.5248 | Reconstruction Loss: 0.4554 | L1 Loss: 0.0694 | l1_alpha: 8.0000e-04 | Tokens: 0 | Self Similarity: -0.0012
Sparsity: 515.2 | Dead Features: 1024 | Total Loss: 0.3679 | Reconstruction Loss: 0.2639 | L1 Loss: 0.1041 | l1_alpha: 8.0000e-04 | Tokens: 0 | Self Similarity: -0.0006
Sparsity: 527.3 | Dead Features: 1024 | Total Loss: 0.7415 | Reconstruction Loss: 0.5943 | L1 Loss: 0.1471 | l1_alpha: 8.0000e-04 | Tokens: 0 | Self Similarity: 0.0013
Sparsity: 518.9 | Dead Features: 1024 | Total Loss: 8.2464 | Reconstruction Loss: 7.6876 | L1 Loss: 0.5588 | l1_alpha: 8.0000e-04 | Tokens: 0 | Self Similari

...

 60%|█████▉    | 33004/55054 [15:14<10:46, 34.08it/s]

Sparsity: 18.9 | Dead Features: 0 | Total Loss: 0.0122 | Reconstruction Loss: 0.0065 | L1 Loss: 0.0057 | l1_alpha: 8.0000e-04 | Tokens: 67584000 | Self Similarity: 0.0004
Sparsity: 34.6 | Dead Features: 0 | Total Loss: 0.0136 | Reconstruction Loss: 0.0085 | L1 Loss: 0.0050 | l1_alpha: 8.0000e-04 | Tokens: 67584000 | Self Similarity: 0.0131
Sparsity: 45.7 | Dead Features: 0 | Total Loss: 0.0192 | Reconstruction Loss: 0.0115 | L1 Loss: 0.0077 | l1_alpha: 8.0000e-04 | Tokens: 67584000 | Self Similarity: 0.0025
Sparsity: 114.1 | Dead Features: 0 | Total Loss: 0.0382 | Reconstruction Loss: 0.0182 | L1 Loss: 0.0200 | l1_alpha: 8.0000e-04 | Tokens: 67584000 | Self Similarity: -0.0055
Sparsity: 118.2 | Dead Features: 0 | Total Loss: 0.0469 | Reconstruction Loss: 0.0231 | L1 Loss: 0.0238 | l1_alpha: 8.0000e-04 | Tokens: 67584000 | Self Similarity: -0.0007
Sparsity: 146.7 | Dead Features: 0 | Total Loss: 0.0699 | Reconstruction Loss: 0.0326 | L1 Loss: 0.0372 | l1_alpha: 8.0000e-04 | Tokens: 6758

 60%|██████    | 33104/55054 [15:17<11:02, 33.15it/s]

Sparsity: 22.4 | Dead Features: 0 | Total Loss: 0.0139 | Reconstruction Loss: 0.0077 | L1 Loss: 0.0062 | l1_alpha: 8.0000e-04 | Tokens: 67788800 | Self Similarity: 0.0002
Sparsity: 37.3 | Dead Features: 0 | Total Loss: 0.0146 | Reconstruction Loss: 0.0094 | L1 Loss: 0.0053 | l1_alpha: 8.0000e-04 | Tokens: 67788800 | Self Similarity: 0.0130
Sparsity: 48.7 | Dead Features: 0 | Total Loss: 0.0201 | Reconstruction Loss: 0.0121 | L1 Loss: 0.0079 | l1_alpha: 8.0000e-04 | Tokens: 67788800 | Self Similarity: 0.0025
Sparsity: 115.9 | Dead Features: 0 | Total Loss: 0.0390 | Reconstruction Loss: 0.0189 | L1 Loss: 0.0201 | l1_alpha: 8.0000e-04 | Tokens: 67788800 | Self Similarity: -0.0054
Sparsity: 121.9 | Dead Features: 0 | Total Loss: 0.0496 | Reconstruction Loss: 0.0248 | L1 Loss: 0.0248 | l1_alpha: 8.0000e-04 | Tokens: 67788800 | Self Similarity: -0.0004
Sparsity: 155.5 | Dead Features: 0 | Total Loss: 0.0742 | Reconstruction Loss: 0.0346 | L1 Loss: 0.0395 | l1_alpha: 8.0000e-04 | Tokens: 6778

 60%|██████    | 33165/55054 [15:19<10:25, 34.98it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [30]:
# Code that actually starts a full training run!

model_name = "EleutherAI/pythia-70m"
dataset_name = "Elriggs/openwebtext-100k" # "Elriggs/openwebtext-100k"
ratio = 8
layers = [0, 1, 2, 3, 4, 5]
wandb_log = False
seed = 0
split = "train"
epoches = 1

setup_execute_training(model_name,
                       dataset_name,
                       ratio,
                       layers,
                       seed,
                       wandb_log=wandb_log,
                       split=split,
                      epoches=epoches)

Activation size: 512


Found cached dataset parquet (/root/.cache/huggingface/datasets/Elriggs___parquet/Elriggs--openwebtext-100k-79076ecafee8a6d5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Loading cached processed dataset at /root/.cache/huggingface/datasets/Elriggs___parquet/Elriggs--openwebtext-100k-79076ecafee8a6d5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-d3de196f0681d12e_*_of_00008.arrow


Number of tokens: 112750592


  0%|          | 3/55054 [00:00<35:28, 25.87it/s]

Sparsity: 2047.0 | Dead Features: 4096 | Total Loss: 0.2620 | Reconstruction Loss: 0.1138 | L1 Loss: 0.1482 | l1_alpha: 8.0000e-04 | Tokens: 0 | Self Similarity: 1.0000
Sparsity: 2022.9 | Dead Features: 4096 | Total Loss: 0.1765 | Reconstruction Loss: 0.0692 | L1 Loss: 0.1073 | l1_alpha: 8.0000e-04 | Tokens: 0 | Self Similarity: -0.0000
Sparsity: 2058.0 | Dead Features: 4096 | Total Loss: 0.6955 | Reconstruction Loss: 0.5341 | L1 Loss: 0.1614 | l1_alpha: 8.0000e-04 | Tokens: 0 | Self Similarity: 0.0009
Sparsity: 2013.3 | Dead Features: 4096 | Total Loss: 0.5138 | Reconstruction Loss: 0.2826 | L1 Loss: 0.2312 | l1_alpha: 8.0000e-04 | Tokens: 0 | Self Similarity: 0.0005
Sparsity: 2054.3 | Dead Features: 4096 | Total Loss: 1.0107 | Reconstruction Loss: 0.6723 | L1 Loss: 0.3384 | l1_alpha: 8.0000e-04 | Tokens: 0 | Self Similarity: 0.0005
Sparsity: 2037.6 | Dead Features: 4096 | Total Loss: 10.2929 | Reconstruction Loss: 8.9718 | L1 Loss: 1.3211 | l1_alpha: 8.0000e-04 | Tokens: 0 | Self Sim

...

Sparsity: 12.8 | Dead Features: 0 | Total Loss: 0.0091 | Reconstruction Loss: 0.0038 | L1 Loss: 0.0053 | l1_alpha: 8.0000e-04 | Tokens: 29696000 | Self Similarity: -0.0145
Sparsity: 36.5 | Dead Features: 0 | Total Loss: 0.0121 | Reconstruction Loss: 0.0071 | L1 Loss: 0.0050 | l1_alpha: 8.0000e-04 | Tokens: 29696000 | Self Similarity: 0.0310
Sparsity: 54.2 | Dead Features: 0 | Total Loss: 0.0174 | Reconstruction Loss: 0.0098 | L1 Loss: 0.0076 | l1_alpha: 8.0000e-04 | Tokens: 29696000 | Self Similarity: -0.0013
Sparsity: 147.9 | Dead Features: 0 | Total Loss: 0.0349 | Reconstruction Loss: 0.0159 | L1 Loss: 0.0190 | l1_alpha: 8.0000e-04 | Tokens: 29696000 | Self Similarity: -0.0799
Sparsity: 141.9 | Dead Features: 0 | Total Loss: 0.0423 | Reconstruction Loss: 0.0205 | L1 Loss: 0.0218 | l1_alpha: 8.0000e-04 | Tokens: 29696000 | Self Similarity: -0.0027
Sparsity: 115.5 | Dead Features: 0 | Total Loss: 0.0864 | Reconstruction Loss: 0.0462 | L1 Loss: 0.0402 | l1_alpha: 8.0000e-04 | Tokens: 29

 27%|██▋       | 14604/55054 [07:14<20:19, 33.16it/s]

Sparsity: 14.9 | Dead Features: 0 | Total Loss: 0.0102 | Reconstruction Loss: 0.0046 | L1 Loss: 0.0056 | l1_alpha: 8.0000e-04 | Tokens: 29900800 | Self Similarity: -0.0145
Sparsity: 39.6 | Dead Features: 0 | Total Loss: 0.0136 | Reconstruction Loss: 0.0083 | L1 Loss: 0.0053 | l1_alpha: 8.0000e-04 | Tokens: 29900800 | Self Similarity: 0.0309
Sparsity: 58.6 | Dead Features: 0 | Total Loss: 0.0193 | Reconstruction Loss: 0.0112 | L1 Loss: 0.0081 | l1_alpha: 8.0000e-04 | Tokens: 29900800 | Self Similarity: -0.0012
Sparsity: 151.1 | Dead Features: 0 | Total Loss: 0.0388 | Reconstruction Loss: 0.0187 | L1 Loss: 0.0201 | l1_alpha: 8.0000e-04 | Tokens: 29900800 | Self Similarity: -0.0798
Sparsity: 153.6 | Dead Features: 0 | Total Loss: 0.0475 | Reconstruction Loss: 0.0233 | L1 Loss: 0.0242 | l1_alpha: 8.0000e-04 | Tokens: 29900800 | Self Similarity: -0.0029
Sparsity: 121.2 | Dead Features: 0 | Total Loss: 0.0948 | Reconstruction Loss: 0.0517 | L1 Loss: 0.0431 | l1_alpha: 8.0000e-04 | Tokens: 29

 27%|██▋       | 14645/55054 [07:15<20:02, 33.60it/s]


KeyboardInterrupt: 

In [ ]:
# Code that actually starts a full training run!

model_name = "EleutherAI/pythia-70m"
dataset_name = "Elriggs/openwebtext-100k" # "Elriggs/openwebtext-100k"
ratio = 1
layers = [0, 1, 2, 3, 4, 5]
wandb_log = False
seed = 0
split = "train"
epoches = 1

setup_execute_training(model_name,
                       dataset_name,
                       ratio,
                       layers,
                       seed,
                       wandb_log=wandb_log,
                       split=split,
                      epoches=epoches)

In [ ]:
with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
# Code that actually starts a full training run!

model_name = "EleutherAI/pythia-70m"
dataset_name = "Elriggs/openwebtext-100k" # "Elriggs/openwebtext-100k"
ratio = 4
layers = [0, 1, 2, 3, 4, 5]
wandb_log = False
seed = 0
split = "train[:50000]"
epoches = 2

setup_execute_training(model_name,
                       dataset_name,
                       ratio,
                       layers,
                       seed,
                       wandb_log=wandb_log,
                       split=split,
                      epoches=epoches)

In [ ]:
# Code that actually starts a full training run!

model_name = "EleutherAI/pythia-70m"
dataset_name = "Elriggs/openwebtext-100k" # "Elriggs/openwebtext-100k"
ratio = 4
layers = [0, 1, 2, 3, 4, 5]
wandb_log = False
seed = 0
split = "train[50000:]"
epoches = 2

setup_execute_training(model_name,
                       dataset_name,
                       ratio,
                       layers,
                       seed,
                       wandb_log=wandb_log,
                       split=split,
                      epoches=epoches)

In [ ]:
# Code that actually starts a full training run!

model_name = "EleutherAI/pythia-160m"
dataset_name = "Elriggs/openwebtext-100k" # "Elriggs/openwebtext-100k"
ratio = 4
layers = [0, 1, 2, 3, 4, 5]
wandb_log = False
seed = 0
split = "train[:50000]"
epoches = 2

setup_execute_training(model_name,
                       dataset_name,
                       ratio,
                       layers,
                       seed,
                       wandb_log=wandb_log,
                       split=split,
                      epoches=epoches)

In [ ]:
# Code that actually starts a full training run!

model_name = "EleutherAI/pythia-160m"
dataset_name = "Elriggs/openwebtext-100k" # "Elriggs/openwebtext-100k"
ratio = 4
layers = [0, 1, 2, 3, 4, 5]
wandb_log = False
seed = 0
split = "train[50000:]"
epoches = 2

setup_execute_training(model_name,
                       dataset_name,
                       ratio,
                       layers,
                       seed,
                       wandb_log=wandb_log,
                       split=split,
                      epoches=epoches)

In [ ]:
# Code that actually starts a full training run!

model_name = "EleutherAI/pythia-410m"
dataset_name = "Elriggs/openwebtext-100k" # "Elriggs/openwebtext-100k"
ratio = 2
layers = [0, 1, 2, 3, 4, 5]
wandb_log = False
seed = 0
split = "train"
epoches = 1

setup_execute_training(model_name,
                       dataset_name,
                       ratio,
                       layers,
                       seed,
                       wandb_log=wandb_log,
                       split=split,
                      epoches=epoches)

Activation size: 1024


Found cached dataset parquet (/root/.cache/huggingface/datasets/Elriggs___parquet/Elriggs--openwebtext-100k-79076ecafee8a6d5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Loading cached processed dataset at /root/.cache/huggingface/datasets/Elriggs___parquet/Elriggs--openwebtext-100k-79076ecafee8a6d5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-62b1708338b4f701_*_of_00008.arrow


Number of tokens: 112750592


  0%|          | 1/55054 [00:00<1:51:54,  8.20it/s]

Sparsity: 1016.9 | Dead Features: 2048 | Total Loss: 0.4088 | Reconstruction Loss: 0.2168 | L1 Loss: 0.1920 | l1_alpha: 8.0000e-04 | Tokens: 0 | Self Similarity: 1.0000
Sparsity: 1031.2 | Dead Features: 2048 | Total Loss: 0.1260 | Reconstruction Loss: 0.0422 | L1 Loss: 0.0837 | l1_alpha: 8.0000e-04 | Tokens: 0 | Self Similarity: -0.0000
Sparsity: 1023.0 | Dead Features: 2048 | Total Loss: 0.1994 | Reconstruction Loss: 0.0835 | L1 Loss: 0.1158 | l1_alpha: 8.0000e-04 | Tokens: 0 | Self Similarity: 0.0009
Sparsity: 1012.4 | Dead Features: 2048 | Total Loss: 0.2024 | Reconstruction Loss: 0.0876 | L1 Loss: 0.1147 | l1_alpha: 8.0000e-04 | Tokens: 0 | Self Similarity: 0.0006
Sparsity: 1037.7 | Dead Features: 2048 | Total Loss: 0.2873 | Reconstruction Loss: 0.1550 | L1 Loss: 0.1324 | l1_alpha: 8.0000e-04 | Tokens: 0 | Self Similarity: 0.0005
Sparsity: 1023.0 | Dead Features: 2048 | Total Loss: 2.1262 | Reconstruction Loss: 1.9278 | L1 Loss: 0.1984 | l1_alpha: 8.0000e-04 | Tokens: 0 | Self Simi

...

  3%|▎         | 1403/55054 [01:42<1:05:46, 13.59it/s]

Sparsity: 23.1 | Dead Features: 0 | Total Loss: 0.0319 | Reconstruction Loss: 0.0192 | L1 Loss: 0.0127 | l1_alpha: 8.0000e-04 | Tokens: 2867200 | Self Similarity: -0.0192
Sparsity: 8.3 | Dead Features: 0 | Total Loss: 0.0115 | Reconstruction Loss: 0.0089 | L1 Loss: 0.0025 | l1_alpha: 8.0000e-04 | Tokens: 2867200 | Self Similarity: -0.0329
Sparsity: 20.2 | Dead Features: 0 | Total Loss: 0.0245 | Reconstruction Loss: 0.0191 | L1 Loss: 0.0054 | l1_alpha: 8.0000e-04 | Tokens: 2867200 | Self Similarity: 0.0224
Sparsity: 20.0 | Dead Features: 0 | Total Loss: 0.0290 | Reconstruction Loss: 0.0237 | L1 Loss: 0.0053 | l1_alpha: 8.0000e-04 | Tokens: 2867200 | Self Similarity: 0.0337
Sparsity: 28.8 | Dead Features: 0 | Total Loss: 0.0383 | Reconstruction Loss: 0.0306 | L1 Loss: 0.0077 | l1_alpha: 8.0000e-04 | Tokens: 2867200 | Self Similarity: 0.0477
Sparsity: 25.6 | Dead Features: 0 | Total Loss: 0.0537 | Reconstruction Loss: 0.0436 | L1 Loss: 0.0101 | l1_alpha: 8.0000e-04 | Tokens: 2867200 | Sel

  3%|▎         | 1503/55054 [01:49<1:04:48, 13.77it/s]

Sparsity: 24.5 | Dead Features: 0 | Total Loss: 0.0328 | Reconstruction Loss: 0.0199 | L1 Loss: 0.0130 | l1_alpha: 8.0000e-04 | Tokens: 3072000 | Self Similarity: -0.0197
Sparsity: 8.0 | Dead Features: 0 | Total Loss: 0.0110 | Reconstruction Loss: 0.0085 | L1 Loss: 0.0025 | l1_alpha: 8.0000e-04 | Tokens: 3072000 | Self Similarity: -0.0312
Sparsity: 20.2 | Dead Features: 0 | Total Loss: 0.0240 | Reconstruction Loss: 0.0185 | L1 Loss: 0.0055 | l1_alpha: 8.0000e-04 | Tokens: 3072000 | Self Similarity: 0.0260
Sparsity: 19.8 | Dead Features: 0 | Total Loss: 0.0279 | Reconstruction Loss: 0.0226 | L1 Loss: 0.0053 | l1_alpha: 8.0000e-04 | Tokens: 3072000 | Self Similarity: 0.0359
Sparsity: 27.4 | Dead Features: 0 | Total Loss: 0.0359 | Reconstruction Loss: 0.0286 | L1 Loss: 0.0073 | l1_alpha: 8.0000e-04 | Tokens: 3072000 | Self Similarity: 0.0534
Sparsity: 25.7 | Dead Features: 0 | Total Loss: 0.0510 | Reconstruction Loss: 0.0409 | L1 Loss: 0.0102 | l1_alpha: 8.0000e-04 | Tokens: 3072000 | Sel

  3%|▎         | 1519/55054 [01:50<1:04:18, 13.88it/s]

# Model + Data

In [3]:
from transformers import AutoModel, AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification, GPTJForSequenceClassification


# Load in the model
def load_model(cfg):
    model = AutoModelForCausalLM.from_pretrained(cfg.model_name)
    model = model.to(cfg.device)
    tokenizer = AutoTokenizer.from_pretrained(cfg.model_name)
    return model, tokenizer

In [4]:
# Download the dataset
# TODO iteratively grab dataset?
def init_dataloader(cfg, model, tokenizer, split="train"):
    cfg.max_length = 256
    token_loader = setup_token_data(cfg, tokenizer, model, seed=cfg.seed, split=split)
    num_tokens = cfg.max_length*cfg.model_batch_size*len(token_loader)
    print(f"Number of tokens: {num_tokens}")
    cfg.total_tokens = num_tokens
    
    return token_loader

In [5]:
# Run 1 datapoint on model to get the activation size

def get_activation_size(cfg, model, tokenizer):
    text = "1"
    tensor_names = [cfg.tensor_name.format(layer=layer) for layer in cfg.layers]
    tokens = tokenizer(text, return_tensors="pt").input_ids.to(cfg.device)
    # Your activation name will be different. In the next cells, we will show you how to find it.
    with torch.no_grad():
        with Trace(model, tensor_names[0]) as ret:
            _ = model(tokens)
            representation = ret.output
            # check if instance tuple
            if(isinstance(representation, tuple)):
                representation = representation[0]
            activation_size = representation.shape[-1]
    print(f"Activation size: {activation_size}")
    cfg.activation_size = activation_size
    return activation_size

In [6]:
# text = "1"
# tokens = tokenizer(text, return_tensors="pt").input_ids.to(cfg.device)
# # Your activation name will be different. In the next cells, we will show you how to find it.
# with torch.no_grad():
#     with Trace(model, 'gpt_neox.layers.0.mlp') as ret:
#         _ = model(tokens)
#         representation = ret.output
#         # check if instance tuple
#         if(isinstance(representation, tuple)):
#             representation = representation[0]
#         activation_size = representation.shape[-1]
# print(f"Activation size: {activation_size}")
# cfg.activation_size = activation_size


In [7]:
# # Set target sparsity to 10% of activation_size if not set

# # NOT USED
# if cfg.sparsity is None:
#     cfg.sparsity = int(activation_size*0.05)
#     print(f"Target sparsity: {cfg.sparsity}")

# target_lower_sparsity = cfg.sparsity * 0.9
# target_upper_sparsity = cfg.sparsity * 1.1
# adjustment_factor = 0.1  # You can set this to whatever you like

# Sparse Autoencocer init

In [8]:
def init_autoencoder(cfg):
    autoencoders = []
    optimizers = []
    for layer in range(len(cfg.layers)):
        params = dict()
        n_dict_components = cfg.activation_size*cfg.ratio # Sparse Autoencoder Size
        params["encoder"] = torch.empty((n_dict_components, cfg.activation_size), device=cfg.device)
        nn.init.xavier_uniform_(params["encoder"])
    
        params["decoder"] = torch.empty((n_dict_components, cfg.activation_size), device=cfg.device)
        nn.init.xavier_uniform_(params["decoder"])
    
        params["encoder_bias"] = torch.empty((n_dict_components,), device=cfg.device)
        nn.init.zeros_(params["encoder_bias"])
    
        params["shift_bias"] = torch.empty((cfg.activation_size,), device=cfg.device)
        nn.init.zeros_(params["shift_bias"])
    
        autoencoder = AnthropicSAE(  # TiedSAE, UntiedSAE, AnthropicSAE
            # n_feats = n_dict_components, 
            # activation_size=cfg.activation_size,
            encoder=params["encoder"],
            encoder_bias=params["encoder_bias"],
            decoder=params["decoder"],
            shift_bias=params["shift_bias"],
        )
        autoencoder.to_device(cfg.device)
        autoencoder.set_grad()
    
        optimizer = torch.optim.Adam(
            [
                autoencoder.encoder, 
                autoencoder.encoder_bias,
                autoencoder.decoder,
                autoencoder.shift_bias,
            ], lr=cfg.lr)
        autoencoders.append(autoencoder)
        optimizers.append(optimizer)
    return autoencoders, optimizers

# Training Run

In [9]:
# tensor_names = ['gpt_neox.layers.0.mlp', 'gpt_neox.layers.5.mlp']

In [10]:
# original_bias = autoencoder.encoder_bias.clone().detach()
# Wandb setup
def setup_wandb(cfg, wandb_run_name):
    secrets = json.load(open("secrets.json"))
    wandb.login(key=secrets["wandb_key"])
    wandb.init(project="Sparse Coding >70m", config=dict(cfg), name=wandb_run_name)
    return wandb_run_name

In [11]:
def training_run(cfg, model, optimizers, autoencoders, token_loader):

    time_since_activation = torch.zeros(autoencoders[0].encoder.shape[0])
    total_activations = torch.zeros(autoencoders[0].encoder.shape[0])
    tensor_names = [cfg.tensor_name.format(layer=layer) for layer in cfg.layers]
    max_num_tokens = cfg.total_tokens # 100_000_000
    save_every = 30_000
    num_saved_so_far = 0

    # Freeze model parameters 
    model.eval()
    model.requires_grad_(False)
    model.to(cfg.device)
    
    last_encoder = autoencoders[0].encoder.clone().detach()
    assert len(cfg.layers) == len(tensor_names), "layers and tensor_names have different lengths"
    for epoch in range(cfg.num_epochs):
        for i, batch in enumerate(tqdm(token_loader)): #,total=int(max_num_tokens/(cfg.max_length*cfg.model_batch_size)))):
            tokens = batch["input_ids"].to(cfg.device)
            # print(f"tokens shape: {tokens.shape}")
            
            with torch.no_grad(): # As long as not doing KL divergence, don't need gradients for model
                
                #print(tensor_names)
                representations = []
                with TraceDict(model, tensor_names) as ret:
                    _ = model(tokens)
                    for tensor_name in tensor_names:
                        representations.append(ret[tensor_name].output)
                    assert not isinstance(representations[0], tuple), "representations is type tuple"
                    # print(len(representations), representations[0].shape)
                    # if(isinstance(representation, tuple)):
                    #     representation = representation[0]
            #print(f"representation is: {representation}")
            #print(f"representation shape is: {representation.shape}")
            
        
            # activation_saver.save_batch(layer_activations.clone().cpu().detach())
            for layer in range(len(cfg.layers)):
                representation = representations[layer]
                layer_activations = rearrange(representation, "b seq d_model -> (b seq) d_model")
                autoencoder = autoencoders[layer]
                optimizer = optimizers[layer]
                
                c = autoencoder.encode(layer_activations)
                x_hat = autoencoder.decode(c)
                
                reconstruction_loss = (x_hat - layer_activations).pow(2).mean()
                l1_loss = torch.norm(c, 1, dim=-1).mean()
                total_loss = reconstruction_loss + cfg.l1_alpha*l1_loss
            
                time_since_activation += 1
                time_since_activation = time_since_activation * (c.sum(dim=0).cpu()==0)
                # total_activations += c.sum(dim=0).cpu()
                if ((i) % 100 == 0): # Check here so first check is model w/o change
                    # self_similarity = torch.cosine_similarity(c, last_encoder, dim=-1).mean().cpu().item()
                    # Above is wrong, should be similarity between encoder and last encoder
                    self_similarity = torch.cosine_similarity(autoencoder.encoder, last_encoder, dim=-1).mean().cpu().item()
                    last_encoder = autoencoder.encoder.clone().detach()
            
                    num_tokens_so_far = i*cfg.max_length*cfg.model_batch_size
                    with torch.no_grad():
                        sparsity = (c != 0).float().mean(dim=0).sum().cpu().item()
                        # Count number of dead_features are zero
                        num_dead_features = (time_since_activation >= min(i, 200)).sum().item()
                    print(f"Sparsity: {sparsity:.1f} | Dead Features: {num_dead_features} | Total Loss: {total_loss:.4f} | Reconstruction Loss: {reconstruction_loss:.4f} | L1 Loss: {cfg.l1_alpha*l1_loss:.4f} | l1_alpha: {cfg.l1_alpha:.4e} | Tokens: {num_tokens_so_far} | Self Similarity: {self_similarity:.4f}")
                    
                    if cfg.wandb_log:
                        wandb.log({
                            'Sparsity': sparsity,
                            'Dead Features': num_dead_features,
                            'Total Loss': total_loss.item(),
                            'Reconstruction Loss': reconstruction_loss.item(),
                            'L1 Loss': (cfg.l1_alpha*l1_loss).item(),
                            'l1_alpha': cfg.l1_alpha,
                            'Tokens': num_tokens_so_far,
                            'Self Similarity': self_similarity
                        })
                    
                    dead_features = torch.zeros(autoencoder.encoder.shape[0])
                    
                    # if(num_tokens_so_far > max_num_tokens):
                    #     print(f"Reached max number of tokens: {max_num_tokens}")
                    #     break
                    
                optimizer.zero_grad()
                total_loss.backward()
                optimizer.step()
    wandb.finish()
        # resample_period = 10000
        # if (i % resample_period == 0):
        #     # RESAMPLING
        #     with torch.no_grad():
        #         # Count number of dead_features are zero
        #         num_dead_features = (total_activations == 0).sum().item()
        #         print(f"Dead Features: {num_dead_features}")
                
        #     if num_dead_features > 0:
        #         print("Resampling!")
        #         # hyperparams:
        #         max_resample_tokens = 1000 # the number of token activations that we consider for inserting into the dictionary
        #         # compute loss of model on random subset of inputs
        #         resample_loader = setup_token_data(cfg, tokenizer, model, seed=i)
        #         num_resample_data = 0
    
        #         resample_activations = torch.empty(0, activation_size)
        #         resample_losses = torch.empty(0)
    
        #         for resample_batch in resample_loader:
        #             resample_tokens = resample_batch["input_ids"].to(cfg.device)
        #             with torch.no_grad(): # As long as not doing KL divergence, don't need gradients for model
        #                 with Trace(model, tensor_names[0]) as ret:
        #                     _ = model(resample_tokens)
        #                     representation = ret.output
        #                     if(isinstance(representation, tuple)):
        #                         representation = representation[0]
        #             layer_activations = rearrange(representation, "b seq d_model -> (b seq) d_model")
        #             resample_activations = torch.cat((resample_activations, layer_activations.detach().cpu()), dim=0)
    
        #             c = autoencoder.encode(layer_activations)
        #             x_hat = autoencoder.decode(c)
                    
        #             reconstruction_loss = (x_hat - layer_activations).pow(2).mean(dim=-1)
        #             l1_loss = torch.norm(c, 1, dim=-1)
        #             temp_loss = reconstruction_loss + cfg.l1_alpha*l1_loss
                    
        #             resample_losses = torch.cat((resample_losses, temp_loss.detach().cpu()), dim=0)
                    
        #             num_resample_data +=layer_activations.shape[0]
        #             if num_resample_data > max_resample_tokens:
        #                 break
    
                    
        #         # sample num_dead_features vectors of input activations
        #         probabilities = resample_losses**2
        #         probabilities /= probabilities.sum()
        #         sampled_indices = torch.multinomial(probabilities, num_dead_features, replacement=True)
        #         new_vectors = resample_activations[sampled_indices]
    
        #         # calculate average encoder norm of alive neurons
        #         alive_neurons = list((total_activations!=0))
        #         modified_columns = total_activations==0
        #         avg_norm = autoencoder.encoder.data[alive_neurons].norm(dim=-1).mean()
    
        #         # replace dictionary and encoder weights with vectors
        #         new_vectors = new_vectors / new_vectors.norm(dim=1, keepdim=True)
                
        #         params_to_modify = [autoencoder.encoder, autoencoder.encoder_bias]
    
        #         current_weights = autoencoder.encoder.data
        #         current_weights[modified_columns] = (new_vectors.to(cfg.device) * avg_norm * 0.02)
        #         autoencoder.encoder.data = current_weights
    
        #         current_weights = autoencoder.encoder_bias.data
        #         current_weights[modified_columns] = 0
        #         autoencoder.encoder_bias.data = current_weights
                
        #         if hasattr(autoencoder, 'decoder'):
        #             current_weights = autoencoder.decoder.data
        #             current_weights[modified_columns] = new_vectors.to(cfg.device)
        #             autoencoder.decoder.data = current_weights
        #             params_to_modify += [autoencoder.decoder]
    
        #         for param_group in optimizer.param_groups:
        #             for param in param_group['params']:
        #                 if any(param is d_ for d_ in params_to_modify):
        #                     # Extract the corresponding rows from m and v
        #                     m = optimizer.state[param]['exp_avg']
        #                     v = optimizer.state[param]['exp_avg_sq']
                            
        #                     # Update the m and v values for the modified columns
        #                     m[modified_columns] = 0  # Reset moving average for modified columns
        #                     v[modified_columns] = 0  # Reset squared moving average for modified columns
            
        #     total_activations = torch.zeros(autoencoder.encoder.shape[0])
        
        
        
    
        # if ((i+2) % save_every ==0): # save periodically but before big changes
        #     model_save_name = cfg.model_name.split("/")[-1]
        #     save_name = f"{model_save_name}_sp{cfg.sparsity}_r{cfg.ratio}_{tensor_names[0]}_ckpt{num_saved_so_far}"  # trim year
    
        #     # Make directory traiend_models if it doesn't exist
        #     import os
        #     if not os.path.exists("trained_models"):
        #         os.makedirs("trained_models")
        #     # Save model
        #     torch.save(autoencoder, f"trained_models/{save_name}.pt")
            
        #     num_saved_so_far += 1
    
        # # Running sparsity check
        # num_tokens_so_far = i*cfg.max_length*cfg.model_batch_size
        # if(num_tokens_so_far > 200000):
        #     if(i % 100 == 0):
        #         with torch.no_grad():
        #             sparsity = (c != 0).float().mean(dim=0).sum().cpu().item()
        #         if sparsity > target_upper_sparsity:
        #             cfg.l1_alpha *= (1 + adjustment_factor)
        #         elif sparsity < target_lower_sparsity:
        #             cfg.l1_alpha *= (1 - adjustment_factor)
        #         # print(f"Sparsity: {sparsity:.1f} | l1_alpha: {cfg.l1_alpha:.2e}")

## Duplicated training run

# Saving

In [12]:
def model_save(cfg, autoencoder, storage_path, filename, layer):
    model_save_name = cfg.model_name.split("/")[-1]

    # start_time = datetime.now().strftime("%Y%m%d-%H%M%S")

    # save_name = f"{model_save_name}_sp{cfg.sparsity}_r{cfg.ratio}_{tensor_names[0]}_{start_time}"  # trim year
    storage_path_name = "trained_models/" + storage_path + f"/layer_{layer}"
    # Make directory traiend_models if it doesn't exist
    if not os.path.exists(storage_path_name):
        os.makedirs(storage_path_name)
    # Save model
    filename = f"L{layer}_{filename}"
    
    torch.save(autoencoder, f"{storage_path_name}/{filename}.pt")
    print(f"Saved file at: {storage_path_name}/{filename}.pt")